In [1]:
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql import SparkSession
import pyspark.sql.functions as sqlFn
import time
from IPython.display import clear_output

In [2]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/28 16:52:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
salesDF = spark.read.format("csv") \
    .option("inferschema","true").option("header","true") \
    .load("sales.csv").withColumn("Sales Id", monotonically_increasing_id())

productsDF = spark.read.format("csv") \
    .option("inferschema","true").option("header","true").load("products.csv")

# Data Format:
salesDF.printSchema()
productsDF.printSchema()

root
 |-- Product Id: string (nullable = true)
 |-- Customer Id: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Sales Id: long (nullable = false)

root
 |-- Product Id: integer (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Product Price: double (nullable = true)



In [4]:
# Sneak peek:
salesDF.show(5)
productsDF.show()

# It's SQL'ing Time!
salesDF.createOrReplaceTempView("sales")
productsDF.createOrReplaceTempView("products")


# joinQuery = """
# select
#     `Sales Id`,
#     products.`Product Id`,
#     `Product Name`,
#     `Customer Id`,
#     `Date`,
#     `Location`,
#     `Source`,
#     `Quantity`,
#     `Product Price`
# from
#     sales full outer join products on sales.`Product Id` = products.`Product Id`
# """
# spark.sql(joinQuery).createOrReplaceTempView("completeTable")

joinedDF = salesDF.join(productsDF, salesDF["Product Id"] == productsDF["Product Id"], "fullouter") \
    .drop(salesDF["Product Id"]) # Product Id column is redundant

+----------+-----------+-------------------+--------+----------+--------+--------+
|Product Id|Customer Id|               Date|Location|    Source|Quantity|Sales Id|
+----------+-----------+-------------------+--------+----------+--------+--------+
|         1|          A|2023-01-01 00:00:00|   India|    Swiggy|       1|       0|
|         2|          A|2022-01-01 00:00:00|   India|    Swiggy|       2|       1|
|         2|          A|2023-01-07 00:00:00|   India|    Swiggy|       3|       2|
|         3|          A|2023-01-10 00:00:00|   India|Restaurant|       1|       3|
|         3|          A|2022-01-11 00:00:00|   India|    Swiggy|       1|       4|
+----------+-----------+-------------------+--------+----------+--------+--------+
only showing top 5 rows

+----------+------------+-------------+
|Product Id|Product Name|Product Price|
+----------+------------+-------------+
|         1|       PIZZA|        100.0|
|         2|     Chowmin|        150.0|
|         3|    sandwich|   

In [5]:
# Anomalies:

# query = """
# select
#     *
# from
#     completeTable
# where
#     `Sales Id` is null or
#     `Product Id` is null or
#     `Product Name` is null or
#     `Customer Id` is null or
#     `Date` is null or
#     `Location` is null or
#     `Source` is null or
#     `Quantity` is null or
#     `Product Price` is null
# """ 
# spark.sql(query).show()

# Cached joinedDF as an action is to be performed 
nullsDF = joinedDF.cache().filter(' is null or '.join(f"`{col}`" for col in joinedDF.columns) + "is null")

nullsDF.show()

+-----------+----+--------+------+--------+--------+----------+------------+-------------+
|Customer Id|Date|Location|Source|Quantity|Sales Id|Product Id|Product Name|Product Price|
+-----------+----+--------+------+--------+--------+----------+------------+-------------+
|       NULL|NULL|    NULL|  NULL|    NULL|     117|      NULL|        NULL|         NULL|
|       NULL|NULL|    NULL|  NULL|    NULL|    NULL|         7|     Boogers|        999.0|
+-----------+----+--------+------+--------+--------+----------+------------+-------------+



In [6]:
# Clean NULL record:

# filter = """
# select * from completeTable
# where `Sales Id` is null or `Sales Id` != 117
# """
# table = spark.sql(filter)

# # To persist/cache or to not?
# table.createOrReplaceTempView("completeTable")

completeDF = joinedDF.filter((joinedDF["Sales Id"]!=117) | (joinedDF["Sales Id"].isNull())).cache()

In [7]:
# Q. Total amount spend by each customer:

# query = """
# select
#     `Customer Id`,
#     sum(ifnull(`Product Price`,0)*`Quantity`) as `Total Spent`,
#     sum(`Quantity`) as `Items Bought`
# from
#     completeTable
# where
#     `Sales Id` is not null
# group by
#     `Customer Id`
# order by
#     `Total Spent` desc
# """
# spark.sql(query).show()

# `Sales Id` is not null -> (Customer Id, Quantity, Product Price) not null
completeDF.where(completeDF["Sales Id"].isNotNull()) \
    .groupBy(completeDF["Customer Id"]) \
    .agg(sqlFn.sum(completeDF["Product Price"]*completeDF["Quantity"]).alias("Total Spent")) \
    .show()

+-----------+-----------+
|Customer Id|Total Spent|
+-----------+-----------+
|          B|    19440.0|
|          C|     6560.0|
|          A|    13830.0|
|          E|    15630.0|
|          D|     4280.0|
+-----------+-----------+



In [8]:
# Q. Total spend on each Product: (Assumption: Product Id 1-to-1 Product Name)

# query = """
# select
#     `Product Name`,
#     sum(ifnull(`Product Price`,0)*ifnull(`Quantity`,0)) as `Total Spent`,
#     sum(ifnull(`Quantity`,0)) as `Units Bought`
# from
#     completeTable
# where
#     `Product Id` is not null
# group by
#     `Product Name`
# order by
#     `Total Spent` desc
# """
# spark.sql(query).show()

completeDF.where(completeDF["Product Id"].isNotNull()) \
    .groupBy(completeDF["Product Name"]) \
    .agg( \
        sqlFn.sum(sqlFn.ifnull(completeDF["Product Price"],sqlFn.lit(0))*sqlFn.ifnull(completeDF["Quantity"],sqlFn.lit(0))) \
        .alias("Total Spent")).show()

+------------+-----------+
|Product Name|Total Spent|
+------------+-----------+
|       PIZZA|     5600.0|
|       Pasta|     3600.0|
|    sandwich|    28560.0|
|     Biryani|     2000.0|
|        Dosa|     3630.0|
|     Boogers|        0.0|
|     Chowmin|    16350.0|
+------------+-----------+



In [9]:
# Q. Total amount of sales in each month:

# query = """
# select
#     year(`Date`) as Year,
#     month(`Date`) as Month,
#     sum(ifnull(`Product Price`,0)*Quantity) as `Total Spent`,
#     sum(`Quantity`) as `Items Bought`
# from
#     completeTable
# where
#     `Date` is not null
# group by
#     year(`Date`), month(`Date`)
# order by
#     year(`Date`) asc, month(`Date`) asc
# """

# spark.sql(query).show()

completeDF.where(completeDF["Sales Id"].isNotNull()) \
    .groupBy(
        sqlFn.year(completeDF["Date"]).alias("Year"),
        sqlFn.month(completeDF["Date"]).alias("Month")
    ) \
    .agg(
        sqlFn.sum(completeDF["Product Price"]*completeDF["Quantity"]).alias("Total Spent"),
        sqlFn.count(completeDF["Sales Id"]).alias("Total Orders"),
        sqlFn.sum(completeDF["Quantity"]).alias("Items Bought")
    ).orderBy("Year","Month").show()

+----+-----+-----------+------------+------------+
|Year|Month|Total Spent|Total Orders|Items Bought|
+----+-----+-----------+------------+------------+
|2022|    1|     1860.0|           5|          13|
|2022|    2|     6470.0|           9|          55|
|2022|    3|      880.0|           3|           7|
|2022|    5|     1890.0|           5|          13|
|2022|    6|     2640.0|           5|          18|
|2022|    7|      950.0|           3|           7|
|2022|   11|     1560.0|           3|          12|
|2023|    1|     6740.0|          18|          50|
|2023|    2|    15680.0|          15|         134|
|2023|    3|      880.0|           5|           8|
|2023|    5|    10910.0|          18|          81|
|2023|    6|     5590.0|          18|          46|
|2023|    7|     1590.0|           5|          15|
|2023|   11|     2100.0|           5|          22|
+----+-----+-----------+------------+------------+



In [27]:
# Q. Yearly Sales:

# query = """
# select
#     year(`Date`) as Year,
#     sum(ifnull(`Product Price`,0)*Quantity) as `Total Spent`,
#     sum(`Quantity`) as `Items Bought`
# from
#     completeTable
# where
#     `Date` is not null
# group by
#     year(`Date`)
# order by
#     year(`Date`) asc
# """
# spark.sql(query).show()


tests = 100
avg = 0
for t in range(1,tests+1):
    start = time.time()


    completeDF.where(completeDF["Sales Id"].isNotNull()) \
    .groupBy(sqlFn.year("Date").alias("Year")) \
    .agg(
        sqlFn.sum(completeDF["Product Price"] * completeDF["Quantity"]).alias("Total Spent"),
        sqlFn.sum("Quantity").alias("Items Bought")
    ).show()
    
    if (t!= tests):
        clear_output()

    end = time.time()
    avg = (avg*(t-1) + (end-start))/t
print("Average time: ",avg)

+----+-----------+------------+
|Year|Total Spent|Items Bought|
+----+-----------+------------+
|2023|    43490.0|         356|
|2022|    16250.0|         125|
+----+-----------+------------+

Average time:  0.275629506111145


In [15]:
# Q. Quarterly Sales:

# query = """
# select
#     year(`Date`) as Year,
#     round((month(`Date`)-1)/4,0)+1 as Quarter,
#     sum(ifnull(`Product Price`,0)*Quantity) as `Total Spent`,
#     sum(Quantity) as `Items Bought`
# from
#     completeTable
# where
#     `Date` is not null
# group by
#     year(`Date`), round((month(`Date`)-1)/4,0)+1
# order by
#     year(`Date`), Quarter
# """

# spark.sql(query).show()

completeDF.where(completeDF["Sales Id"].isNotNull()) \
    .groupBy(
        sqlFn.year("Date").alias("Year"), 
        (sqlFn.round((sqlFn.month("Date")-sqlFn.lit(1)) / sqlFn.lit(4),0) + 1).alias("Quarter")
    ) \
    .agg(
        sqlFn.sum(completeDF["Product Price"] * completeDF["Quantity"]).alias("Total Spent"),
        sqlFn.sum("Quantity").alias("Items Bought")
    ).orderBy("Year","Quarter").show()

+----+-------+-----------+------------+
|Year|Quarter|Total Spent|Items Bought|
+----+-------+-----------+------------+
|2022|    1.0|     8330.0|          68|
|2022|    2.0|     5410.0|          38|
|2022|    3.0|      950.0|           7|
|2022|    4.0|     1560.0|          12|
|2023|    1.0|    22420.0|         184|
|2023|    2.0|    17380.0|         135|
|2023|    3.0|     1590.0|          15|
|2023|    4.0|     2100.0|          22|
+----+-------+-----------+------------+



In [13]:
# Q. Total number of orders by each category: 

# query = """
# select
#     `Product Id`,
#     `Product Name`,
#     count(`Sales Id`) as `Orders Placed`
# from
#     completeTable
# where
#     `Product Id` is not null
# group by
#     `Product Id`,`Product Name`
# """
# spark.sql(query).show()

completeDF.where(completeDF["Sales Id"].isNotNull()) \
    .groupBy(
        sqlFn.year("Date").alias("Year"), 
        (sqlFn.round((sqlFn.month("Date")-sqlFn.lit(1)) / sqlFn.lit(4),0) + 1).alias("Quarter")
    ) \
    .agg(
        sqlFn.sum(completeDF["Product Price"] * completeDF["Quantity"]).alias("Total Spent"),
        sqlFn.sum("Quantity").alias("Items Bought")
    ).orderBy("Year","Quarter").show()

+----------+------------+-------------+
|Product Id|Product Name|Orders Placed|
+----------+------------+-------------+
|         1|       PIZZA|           21|
|         6|       Pasta|            6|
|         3|    sandwich|           48|
|         5|     Biryani|            6|
|         4|        Dosa|           12|
|         7|     Boogers|            0|
|         2|     Chowmin|           24|
+----------+------------+-------------+



In [22]:
# Q. Top 5 ordered items:

# query = """
# select
#     `Product Id`,
#     `Product Name`,
#     ifnull(sum(Quantity),0) as `Items Bought`,
#     count(`Sales Id`) as Orders
# from
#     completeTable
# where
#     `Product Id` is not null
# group by
#     `Product Id`, `Product Name`
# order by
#     `Items Bought` desc
# limit
#     5
# """
# spark.sql(query).show()

completeDF.where(completeDF["Product Id"].isNotNull()) \
    .groupBy(completeDF["Product Id"]) \
    .agg(
        sqlFn.max("Product Name").alias("Product Name"), 
        sqlFn.sum("Quantity").alias("Items Bought"),
        sqlFn.count(completeDF["Sales Id"]).alias("Total Orders")      
    ).orderBy(["Total Orders"],ascending=[0]).limit(5).show() # Order by "Items Bought" or "Total Orders"

+----------+------------+------------+------------+
|Product Id|Product Name|Items Bought|Total Orders|
+----------+------------+------------+------------+
|         3|    sandwich|         238|          48|
|         2|     Chowmin|         109|          24|
|         1|       PIZZA|          56|          21|
|         4|        Dosa|          33|          12|
|         5|     Biryani|          25|           6|
+----------+------------+------------+------------+



In [24]:
# Q. Frequency of Customer visit:

# query = """
# select
#     `Customer Id`,
#     count(`Sales Id`) as `Frequency of Purchases`
# from
#     completeTable
# where
#     `Customer Id` is not null
# group by
#     `Customer Id`
# """
# spark.sql(query).show()

completeDF.where(completeDF["Customer Id"].isNotNull()) \
    .groupBy("Customer Id") \
    .agg(
        sqlFn.count(completeDF["Sales Id"]).alias("Frequency of Purchases")
    ).show()

+-----------+----------------------+
|Customer Id|Frequency of Purchases|
+-----------+----------------------+
|          B|                    36|
|          C|                    18|
|          A|                    33|
|          E|                    18|
|          D|                    12|
+-----------+----------------------+



In [25]:
# Q. Total sales by each country:

# query = """
# select
#     `Location`,
#     sum(ifnull(`Product Price`,0)*`Quantity`) as `Country Sales Amount`
# from
#     completeTable
# where
#     `Location` is not null
# group by
#      `Location`
# """
# spark.sql(query).show()

completeDF.where(completeDF["Sales Id"].isNotNull()) \
    .groupBy("Location") \
    .agg(
        sqlFn.sum(completeDF["Product Price"]*completeDF["Quantity"]).alias("Country Sales Amount")
    ).show()

+--------+--------------------+
|Location|Country Sales Amount|
+--------+--------------------+
|   India|             19600.0|
|     USA|              7310.0|
|      UK|             32830.0|
+--------+--------------------+



In [26]:
# Q. Total sales by order source:

# query = """
# select
#     `Source`,
#     sum(ifnull(`Product Price`,0)*`Quantity`) as `Source Sales Amount`,
#     count(`Sales Id`) as Orders
# from
#     completeTable
# where
#     `Source` is not null
# group by
#      `Source`
# """

# spark.sql(query).show()

completeDF.where(completeDF["Sales Id"].isNotNull()) \
    .groupBy("Source") \
    .agg(
        sqlFn.sum(completeDF["Product Price"]*completeDF["Quantity"]).alias("Source Sales Amount")
    ).show()

+----------+-------------------+
|    Source|Source Sales Amount|
+----------+-------------------+
|    Swiggy|            20260.0|
|Restaurant|            18580.0|
|    zomato|            20900.0|
+----------+-------------------+

